In [2]:
#pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pillow

                                              0.0/2.5 MB ? eta -:--:--
     --                                       0.1/2.5 MB 8.3 MB/s eta 0:00:01
     ------                                   0.4/2.5 MB 6.3 MB/s eta 0:00:01
     ----------                               0.6/2.5 MB 5.8 MB/s eta 0:00:01
     --------------                           0.9/2.5 MB 5.6 MB/s eta 0:00:01
     -----------------                        1.1/2.5 MB 5.1 MB/s eta 0:00:01
     ---------------------                    1.4/2.5 MB 5.4 MB/s eta 0:00:01
     -------------------------                1.6/2.5 MB 5.1 MB/s eta 0:00:01
     -----------------------------            1.8/2.5 MB 5.3 MB/s eta 0:00:01
     --------------------------------         2.0/2.5 MB 5.2 MB/s eta 0:00:01
     ------------------------------------     2.3/2.5 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 5.2 MB/s eta 0

In [1]:
#importing labriries
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
#disable the use of OpenCL in OpenCV
cv2.ocl.setUseOpenCL(False)

## preparing the train data:

In [3]:
#flow_from_directory isusing for a data like the one i have with diffrent file for each class
train_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1, validation_split=0.2).flow_from_directory(
        'data2/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',subset='training')

Found 23769 images belonging to 7 classes.


## preparing the test data:

In [4]:
val_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1,validation_split=0.2).flow_from_directory(
        'data2/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',subset='validation')


Found 5941 images belonging to 7 classes.


In [5]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

## building the model :

In [6]:

my_mental_health = Sequential([#features extraction phase
layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),#conv2D convolution over images/input with 1 color(greyscale)
layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),#relu is activation function that turn the negative to zero
layers.MaxPooling2D(pool_size=(2, 2)),#maxpooling2d condensing the data (take  the max from the convolution map) to reduice the size
layers.Dropout(0.25),#protect from overfitting
layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Dropout(0.25),
layers.Flatten(),#flatten is preparing the data to the neural network
layers.Dense(1024, activation='relu'),#1024 number of nerons
layers.Dropout(0.5),
layers.Dense(7, activation='softmax')])#7units corresponding to the number of classes
#softmax is used to normalize the output and gives probabilities of each class


In [7]:
my_mental_health.compile(loss='categorical_crossentropy', optimizer=Adam( learning_rate=0.0001), metrics=['accuracy'])
#categorical_crossentropy used for multi-class classification 

In [8]:
my_mental_health.summary()# to show the steps of the model for the data

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

## fitting the model :

In [9]:
my_mental_health_info = my_mental_health.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // 64,
        epochs=100,
        validation_data=val_generator,
        validation_steps=val_generator.samples // 64,callbacks=[early_stopping])

Epoch 1/100
371/371 [==============================] - 234s 626ms/step - loss: 1.8160 - accuracy: 0.2400 - val_loss: 1.7994 - val_accuracy: 0.2626
Epoch 2/100
371/371 [==============================] - 73s 196ms/step - loss: 1.7668 - accuracy: 0.2681 - val_loss: 1.7511 - val_accuracy: 0.2879
Epoch 3/100
371/371 [==============================] - 72s 194ms/step - loss: 1.7281 - accuracy: 0.3003 - val_loss: 1.7005 - val_accuracy: 0.3077
Epoch 4/100
371/371 [==============================] - 71s 190ms/step - loss: 1.6766 - accuracy: 0.3317 - val_loss: 1.6502 - val_accuracy: 0.3516
Epoch 5/100
371/371 [==============================] - 74s 199ms/step - loss: 1.6298 - accuracy: 0.3574 - val_loss: 1.6047 - val_accuracy: 0.3725
Epoch 6/100
371/371 [==============================] - 78s 211ms/step - loss: 1.5892 - accuracy: 0.3782 - val_loss: 1.5642 - val_accuracy: 0.3843
Epoch 7/100
371/371 [==============================] - 76s 205ms/step - loss: 1.5508 - accuracy: 0.3995 - val_loss: 1.5420 


## make predictions

In [25]:
test_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1).flow_from_directory(
        'data2/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical', shuffle=False)

Found 7178 images belonging to 7 classes.


In [26]:
test_loss, test_accuracy = my_mental_health.evaluate(test_generator, steps= test_generator.samples// 64)

112/112 [==============================] - 11s 94ms/step - loss: 1.0413 - accuracy: 0.6150


## check accuracy

In [27]:
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Loss: 1.0413
Test Accuracy: 0.6150


In [13]:
predictions = my_mental_health.predict(test_generator)

113/113 [==============================] - 47s 420ms/step


In [28]:
import numpy as np
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the predicted class with highest probability
true_labels = test_generator.classes  # Get the true labels of the test samples

accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy:", accuracy)

Accuracy: 0.6060183895235441


## save the model 

In [15]:
my_mental_health.save('my_mental_health2.h5')